In [1]:
import os
import sys
sys.path.append('../')
sys.path.append('../datasets')
sys.path.append('../models')
import torch
import torchvision
import torch.backends.cudnn as cudnn
import random
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from importlib import reload
from datasets.three_dim_shapes import ThreeDimShapesDataset
from datasets.small_norb import SmallNORBDataset
from datasets.seq_mnist import SequentialMNIST
import models.seqae as seqae
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from datasets import so3_data as so3d
from einops import rearrange
from sklearn.metrics import r2_score
import pdb
from einops import rearrange

import copy

import csv
import ast
from source import yaml_utils as yu


from tqdm import tqdm

if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
else:
    device = torch.device('cpu')
    gpu_index = -1

    
    
from utils import yaml_utils as yu
import yaml
rootpath = '/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/'
result_dir = '/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result'



os.listdir(result_dir)

['20220617_default_run_mnist_bg',
 '20220624_Mstar_longer_tp_neuralM_comm',
 '20220621_NeuralMstar_bg_neuralM_vanilla',
 '20220624_Mstar_longer_tp_neuralM_vanilla',
 '20220630_NeuralMstar_comm_0',
 '20220617_NeuralMstar_bg_neuralM_comm',
 '20220628_NeuralMstar_comm_0',
 '20220621_NeuralMstar_bg_neuralM_comm',
 '20220621_so3_rcst_so3run',
 '20220615_default_run_mnist',
 '20220615_NeuralMstar',
 '20220615_NeuralMstar_neuralM_vanilla',
 '20220628_pfkube_try_0',
 '20220701_so3_various_dat_three_0',
 '20220630_longer_tp_0',
 '20220628_so3_idnet_0',
 '20220611_so3_so3run_rcst',
 '20220628_so3_various_dat_0',
 '20220701_so3_various_dat_two_0',
 '20220611_so3_so3run_rcst_iter',
 '20220701_so3_various_dat_four_0',
 '20220624_NeuralMstar_comm_bg_neuralM_vanilla',
 '20220628_so3_linear_net_0',
 '20220624_NeuralMstar_comm_bg_neuralM_comm',
 '20220630_so3_various_dat_0',
 '20220511_so3_exp_0',
 'temp',
 '20220629_so3_various_dat_0',
 '20220628_longer_tp_0',
 '20220611_so3_so3run',
 '20220611_so3',


In [3]:

datamode='MLP'
#query = 'dat_sphere_Linearpt'
query = f"""dat_sphere_{datamode}pt"""
targlist = filter_list(query, os.listdir(exp_result))
targlist

['data_filenameso3dat_sphere_MLPpt_nameSeqAELSTSQ_LinearNet',
 'data_filenameso3dat_sphere_MLPpt_nameSeqAELSTSQ_so3Net']

In [7]:
results = {} 
inferred_Ms = {} 
tp = 3
n_cond = 5


for targdir in targlist:
    
    Mlist = [] 
    targdir_path = os.path.join(exp_result, targdir)
    config = load_config(targdir_path)
    
#     dataconfig = config['train_data']
#     dataconfig['args']['T'] = tp + n_cond

    dataconfig = config['train_data']
    original_datfile = dataconfig['args']['data_filename']
    trans_datfile = original_datfile.split('.')[0] + '_shared_trans.pt'  
    dataconfig['args']['data_filename'] = trans_datfile   
    dataconfig['args']['T'] = tp + n_cond
    
    data = yu.load_component(dataconfig)
    train_loader = DataLoader(data, 
                              batch_size=config['batchsize'],
                              shuffle=True,
                              num_workers=config['num_workers'])
    model = yu.load_component(config['model'])
    iterlist = iter_list(targdir_path)
    
    if len(iterlist) == 0:
        print(f"""There is no model trained for {targdir_path}""")
    else:
        maxiter = np.max(iter_list(targdir_path))
        load_model(model, targdir_path, maxiter)
        model = model.eval().to(device)

        # Initialize lazy modules
        images = iter(train_loader).next()
        images = images.to(device)
        model(images[:, :2])

        with torch.no_grad():
            scores = [] 
            for images in tqdm(train_loader):
                images = images.to(device)
                images_cond = images[:, :n_cond]
                images_target = images[:, n_cond:n_cond+tp]
                M = model.get_M(images_cond) #n a a
                H = model.encode(images_cond[:, -1:])[:, 0] # n s a

                xs = []
                for r in range(tp):
                    H = H @ M 
                    x_next_t = model.decode(H[:, None])
                    xs.append(x_next_t)

                x_next = torch.cat(xs, axis=1)
                r2_losses = [] 
                for k in range(tp):
                    r2_loss_t = r2_score(images_target[:,k].to('cpu').numpy(), x_next[:,k].to('cpu').numpy()) 
                    r2_losses.append(r2_loss_t)
                scores.append(torch.tensor(r2_losses))
                Mlist.append(M)
        
                
        Mlist = torch.cat(Mlist)       
        scores = torch.stack(scores)
        av_score = torch.mean(scores, axis=0)
        av_var = torch.var(scores, axis=0)
        results[targdir] = [av_score, av_var]
        inferred_Ms[targdir] = Mlist 


    

snapshot_model_iter_1000000


/usr/local/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
100%|██████████| 313/313 [00:01<00:00, 227.72it/s]
/usr/local/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


snapshot_model_iter_1000000


100%|██████████| 313/313 [00:01<00:00, 189.75it/s]


In [8]:
results

{'data_filenameso3dat_sphere_MLPpt_nameSeqAELSTSQ_LinearNet': [tensor([0.6080, 0.3566, 0.2411], dtype=torch.float64),
  tensor([0.0007, 0.0019, 0.0020], dtype=torch.float64)],
 'data_filenameso3dat_sphere_MLPpt_nameSeqAELSTSQ_so3Net': [tensor([0.6687, 0.4831, 0.3950], dtype=torch.float64),
  tensor([0.0007, 0.0017, 0.0023], dtype=torch.float64)]}